In [1]:
import numpy as np
from matplotlib import pylab as plt
%matplotlib inline
plt.rcParams['text.usetex'] = True
import scipy
from scipy import integrate as sciint
from scipy import optimize as sciopt
from scipy.special import kn
import numba as nb
import pickle
import funcs

c=2.9792458e10#cm s-1
mev2erg = 1.0218e-6
mpc2 = 938.72*mev2erg # erg
pi = np.pi
day2sec=5184000
Mpc2cm = 3.086e+24
cgi2mJy = 1.e26

n0 = 1.e-3 #cm-3
eps_e = 0.5
eps_b = 0.5
theta_v = 0.25
theta_j = 0.3
D_L = 41*Mpc2cm
Gshmax = 100
theta_G = 0.059 # Gaussian sigma for the jet
theta_c = 0.3 # JET TRUNCATION ANGLE
E_g = 1.16e52


In [2]:
phi=1
T=1*day2sec
nu = 3e9


In [3]:
phi = np.random.random()*pi
print(phi)
%timeit funcs.integral_theta(phi, T, nu, eps_e, eps_b, Gshmax, n0, theta_v, theta_G, theta_c, E_g)

2.836512991210865
612 ms ± 14.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
import cProfile
cProfile.run('funcs.integral_theta(phi, T, nu, eps_e, eps_b, Gshmax, n0, theta_v, theta_G, theta_c, E_g)', sort=2)

         322868 function calls (305765 primitive calls) in 0.708 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.708    0.708 {built-in method builtins.exec}
        1    0.000    0.000    0.708    0.708 <string>:1(<module>)
        1    0.010    0.010    0.708    0.708 {funcs.integral_theta}
      600    0.003    0.000    0.682    0.001 _root_scalar.py:61(root_scalar)
      300    0.022    0.000    0.611    0.002 _zeros_py.py:94(newton)
     6172    0.010    0.000    0.374    0.000 _quadpack_py.py:23(quad)
     6172    0.005    0.000    0.359    0.000 _quadpack_py.py:505(_quad)
     6172    0.354    0.000    0.354    0.000 {built-in method scipy.integrate._quadpack._qagse}
     5697    0.005    0.000    0.229    0.000 <__array_function__ internals>:2(isclose)
23107/6004    0.021    0.000    0.225    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
     5697   

In [2]:
def gaussian_profile(theta, theta_G, theta_c, E_g):
    res = np.zeros_like(theta)
    mask = theta<=theta_c
    res[mask] = E_g * np.exp(-0.5*(theta[mask]/theta_G)**2)
    return res

def U(t, x):
    u = 1/mpc2/c**3
    return u * x * t**(-3)

# _BM = C_BM^2*t^-3 (eq.1 & 2)
def BM(t, x):
    bm = 17/8/pi
    return bm * U(t, x)

# _ST = C_ST^2*t^-6/5
def ST(t, x):
    st = (2*1.15/5)**2
    return st * U(t, x)**(2/5)

def A(t, x):
    return BM(t, x) + ST(t, x)

#Gamma_shell^2 * beta_shell^2
def GB2(t, x, Gshmax) :
    val = A(t, x)
    val[val>=Gshmax**2] = Gshmax**2
    return val

#Gamma_shell^2
def G2(t, x, Gshmax):
    return GB2(t, x, Gshmax) + 1


def integrand(theta_array, phi):
    eg = funcs.gaussian_profile(theta_array, theta_G, theta_c, E_g)
    mu_array = np.cos(theta_array)*np.cos(theta_v) + np.sin(theta_array)*np.sin(theta_v)*np.cos(phi)
    t_array = funcs.find_emission_time(T, eg/n0, Gshmax, mu_array)
    gb2 = GB2(t_array, eg/n0, Gshmax)
    gsh = np.sqrt(gb2+1)
    bsh = np.sqrt(1-1/gsh**2)
    Roc = funcs.R_over_c(t_array, eg/n0, Gshmax)
    
    res = abs(mu_array-bsh)/(1-mu_array*bsh) * Roc**2

    g,b,p,eprime,nprime = funcs.shocked_medium_array(gsh, n0)
    eps_prime, alpha_prime, delta = funcs.emissivity_array(nu, bsh, t_array, mu_array, g, b, p, eprime, nprime, eps_e, eps_b)
    
    res *= eps_prime
    res /= alpha_prime
    res /= delta**3
    
    delta_s = Roc*c / 12. / g**2 / abs(mu_array - bsh)
    tau = alpha_prime * delta * delta_s
    extinction = 1-np.exp(-tau)
    extinction[extinction<1.e-6] = tau[extinction<1.e-6]
    res *= extinction

    res *= np.sin(theta_array)

    return res

In [3]:
%timeit funcs.integral_phi(T,nu, eps_e, eps_b, Gshmax, n0, theta_v, theta_G, theta_c, E_g)

10.7 s ± 238 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
len(phi_array),len(np.arange(0, theta_c, 0.001))

NameError: name 'phi_array' is not defined

In [3]:
def integral_theta(phi, T, nu):
    theta = np.arange(0, theta_c, 0.001)
    res = integrand(theta, phi)
    mid_points = (res[1:]+res[:-1])/2
    integral = (mid_points*np.diff(theta_array)).sum()
    return integral

In [5]:
def integrand_phi(phi_array, T, nu):
    integrand2 = np.zeros_like(phi_array)
    for i,phi in enumerate(phi_array):
        integrand2[i] = integral_theta(phi, T, nu)
    return integrand2

In [6]:
def integral_phi(T,nu):
    phi_array = np.arange(0, pi, 0.1) #function is even in phi
    integrand = integrand_phi(phi_array, T, nu)
    mid_points = (integrand[1:]+integrand[:-1])/2
    integral2 = (mid_points*np.diff(phi_array)).sum()
    return 2*integral2    #times 2 because even

In [7]:
phi_array = np.arange(0, pi, 0.1)
y = integrand_phi(phi_array, T, nu)

NameError: name 'theta_array' is not defined

In [ ]:
plt.plot(phi_array, y)

In [ ]:
%timeit integral_phi(T,nu)

In [ ]:
plt.plot(theta_array, res)

In [ ]:
plt.plot(theta_array, integrand() )

In [ ]:


g,b,p,eprime,nprime = funcs.shocked_medium_array(gsh, n0)
eps_prime, alpha_prime, delta = funcs.emissivity_array(nu, bsh, t_array, mu_array, g, b, p, eprime, nprime, eps_e, eps_b)

In [ ]:
res *= eps_prime    
res /= alpha_prime
res /= delta**3

In [ ]:
delta_s = Roc*c / 12. / g**2 / abs(mu_array - bsh)
tau = alpha_prime * delta * delta_s
extinction = 1-np.exp(-tau)
extinction[extinction<1.e-6] = tau[extinction<1.e-6]
res *= extinction

res *= np.sin(theta_array)


In [ ]:
mid_points = (res[1:]+res[:-1])/2
integral1 = (mid_points*np.diff(theta_array)).sum()
print(integral1)

In [ ]:
def emission_time(theta, phi, T, x, Gshmax, theta_v):
    theta = np.asarray(theta)
    phi = np.asarray(phi)
    mu = np.cos(theta)*np.cos(theta_v) + np.sin(theta)*np.sin(theta_v)*np.cos(phi)
    
def first_integral(phi, T, nu):
    theta_array = np.arange(0, theta_j, 0.005)
    x_array = np.zeros_like(theta_array)
    t_array = np.zeros_like(theta_array)
    res_array = np.zeros_like(theta_array)
    mu_array = np.cos(theta_array)*np.cos(theta_v) + np.sin(theta_array)*np.sin(theta_v)*np.cos(phi)
    for i,theta in enumerate(theta_array):
        mu = mu_array[i]
        x_array[i] = funcs.gaussian_profile(theta_array[i], theta_G, theta_c, E_g)/n0
        t_array[i] = funcs.find_emission_time(T, x_array[i], Gshmax, mu)
        gb2 = funcs.GB2(t_array[i], x_array[i], Gshmax)
        gsh = np.sqrt(gb2+1)
        bsh = np.sqrt(1-1/gsh**2)
        Roc = funcs._R_over_c(t_array[i], x_array[i], Gshmax)
        
        res_array[i] = abs(mu-bsh)/(1-mu*bsh) * Roc**2
        
        g,b,p,eprime,nprime = funcs.shocked_medium(gsh, n0)
        eps_prime, alpha_prime, delta = funcs.emissivity(nu, bsh, t_array[i], mu_array[i], g, b, p, eprime, nprime, eps_e, eps_b)
        
        res_array[i] *= eps_prime    
        res_array[i] /= alpha_prime
        res_array[i] /= delta**3
    
        #internal opacity
        #B39
        delta_s = Roc*c / 12. / g**2 / abs(mu - bsh)
        tau = alpha_prime * delta * delta_s

        if np.exp(-tau)!=1:
            res_array[i] *= (1-np.exp(-tau))
        else: #case where tau is too small and the exp numerically resolve into exactly 1.
            res_array[i] *= tau
    sinth = np.sin(theta_array)
    integrand = res_array*sinth
    mid_points = (integrand[1:]+integrand[:-1])/2
    integral1 = (mid_points*np.diff(theta_array)).sum()
    return integral1

In [ ]:
def second_integral(T, nu):
    phi_array = np.arange(0, pi, 0.1) #function is even in phi
    integrand2 = np.zeros_like(phi_array)
    for i,phi in enumerate(phi_array):
        integrand2[i] = first_integral(phi, T, nu)
    mid_points = (integrand2[1:]+integrand2[:-1])/2
    integral2 = (mid_points*np.diff(phi_array)).sum()
    return 2*integral2    #times 2 because even

In [ ]:
nu = 3e9
T_arr = np.array([0.1, 1, 10, 100])*day2sec
flux = np.zeros_like(T_arr)
for i,T in enumerate(T_arr):
    flux[i] = second_integral(T,nu)*c**2/4/pi/D_L#/cgi2mJy